In [1]:
data_var = '2024-02-13'

In [2]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [3]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [4]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
11013,2024-02-13,Eua Nba,00:30,Los Angeles Clippers,Minnesota Timberwolves,1.53,2.61,223.5,1.85,2.09,-5.5,2.10,1.83,zNML1xWc,0.653595,0.383142,0.540541,0.478469,0.036737,0.6,0.4,NaN,NaN,166.936,47.794496,0.286304,192.178,54.743575,0.284859,155.292,192.266,119.84,263.16,0.0,0.0,0.0,0.0,0.368925,0.086145,0.097160,0.08,0.016,33.125000,0.767157,0.9,0.132843,0.84,0.168,9.583333,0.667758,0.7,0.032242
11014,2024-02-13,Eua Nba,21:30,Brooklyn Nets,Boston Celtics,3.94,1.28,228.5,1.81,2.07,8.5,2.00,1.94,EDLP0do4,0.253807,0.781250,0.552486,0.483092,0.035057,0.4,0.2,NaN,NaN,239.786,67.883344,0.283100,150.966,16.573790,0.109785,232.336,167.956,170.97,158.40,0.0,0.0,0.0,0.0,0.720653,0.094767,0.021536,-1.68,-0.336,-8.750000,0.538498,0.3,-0.238498,2.08,0.416,0.673077,0.666295,0.8,0.133705
11015,2024-02-13,Eua Nba,21:30,Orlando Magic,Oklahoma City Thunder,2.23,1.70,225.5,1.83,2.04,1.5,2.10,1.84,8zLTaG0A,0.448430,0.588235,0.546448,0.490196,0.036666,0.6,0.4,NaN,NaN,198.398,75.088449,0.378474,178.870,31.440428,0.175773,191.808,183.306,158.46,193.14,0.0,0.0,0.0,0.0,0.190721,0.076740,0.093324,1.74,0.348,3.534483,0.532018,0.6,0.067982,-1.63,-0.326,-2.147239,0.583159,0.5,-0.083159
11016,2024-02-13,Alemanha Bbl,14:30,Braunschweig,Syntainics MBC,1.33,3.38,173.5,1.82,1.94,-8.5,2.01,1.76,pSMHsqZk,0.751880,0.295858,0.549451,0.515464,0.047738,0.6,0.6,NaN,NaN,166.396,58.065636,0.348961,303.388,163.389600,0.538550,187.746,463.330,265.00,240.24,0.0,0.0,0.0,0.0,0.615528,0.045134,0.093781,3.80,0.760,0.434211,0.000000,0.0,0.000000,14.39,2.878,0.826963,0.000000,0.0,0.000000
11017,2024-02-13,Alemanha Bbl,16:00,Alba Berlin,Gottingen,1.06,9.40,180.5,1.80,1.96,-16.5,2.00,1.77,QgKaWUm3,0.943396,0.106383,0.555556,0.510204,0.049779,0.8,0.8,NaN,NaN,138.534,18.717967,0.135115,244.408,76.533143,0.313137,113.506,277.136,131.32,342.00,0.0,0.0,0.0,0.0,1.127585,0.060179,0.086278,0.11,0.022,2.727273,0.000000,0.0,0.000000,-0.14,-0.028,-300.000000,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11081,2024-02-13,Eua Ncaa,23:00,San Diego State,Colorado State,1.41,3.03,139.5,1.80,1.95,-6.5,2.00,1.71,ETkQMVIh,0.709220,0.330033,0.555556,0.512821,0.039253,0.4,0.2,NaN,NaN,112.114,24.282210,0.216585,103.364,25.980780,0.251352,93.374,123.370,117.45,100.01,0.0,0.0,0.0,0.0,0.515997,0.056569,0.110545,1.13,0.226,1.814159,0.810688,1.0,0.189312,-3.63,-0.726,-2.796143,0.000000,0.0,0.000000
11082,2024-02-13,Eua Ncaa,23:00,Virginia Tech,Florida State,1.47,2.79,150.5,1.80,1.95,-6.5,2.00,1.71,8zsV8Ovf,0.680272,0.358423,0.555556,0.512821,0.038695,0.4,1.0,NaN,NaN,132.900,34.046119,0.256178,173.546,43.314851,0.249587,137.374,172.342,162.14,135.24,0.0,0.0,0.0,0.0,0.438207,0.056569,0.110545,-0.25,-0.050,-9.400000,0.674165,0.7,0.025835,2.49,0.498,3.594378,0.000000,0.0,0.000000
11083,2024-02-13,Eua Ncaa,23:00,Wisconsin,Ohio State,1.23,4.44,139.5,1.91,1.91,-10.5,2.10,1.67,rTLGmRIi,0.813008,0.225225,0.523560,0.523560,0.038233,0.8,0.6,NaN,NaN,109.286,23.385546,0.213985,126.134,20.339984,0.161257,106.638,119.956,132.84,126.44,0.0,0.0,0.0,0.0,0.800639,0.000000,0.161303,1.60,0.320,0.718750,0.753531,0.9,0.146469,-3.86,-0.772,-4.455959,0.000000,0.0,0.000000
11084,2024-02-13,Europa Liga Letônia,14:00,Ventspils,Valmiera Glass Via,1.01,12.80,172.5,1.85,1.85,-20.5,

## Apostas do dia

### Over v0 (LDA)

In [8]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,00:30,Eua Nba,Los Angeles Clippers,Minnesota Timberwolves,223.5,1.85,1.0000,Over
1,21:30,Eua Nba,Brooklyn Nets,Boston Celtics,228.5,1.81,1.0000,Over
2,21:30,Eua Nba,Orlando Magic,Oklahoma City Thunder,225.5,1.83,0.9999,Over
3,22:00,Eua Nba,Milwaukee Bucks,Miami Heat,221.5,1.88,1.0000,Over
